
## Description
We read in a file of metropolitan regions and associated sports teams from [assets/wikipedia_data.html](assets/wikipedia_data.html) and answer some questions about each metropolitan region. Each of these regions may have one or more teams from the "Big 4": NFL (football, in [assets/nfl.csv](assets/nfl.csv)), MLB (baseball, in [assets/mlb.csv](assets/mlb.csv)), NBA (basketball, in [assets/nba.csv](assets/nba.csv) or NHL (hockey, in [assets/nhl.csv](assets/nhl.csv)).

For each sport we would like to answer the question: **what is the win/loss ratio's correlation with the population of the city it is in?** Win/Loss ratio refers to the number of wins over the number of wins plus the number of losses. To calculate the correlation with [`pearsonr`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.pearsonr.html), so we are going to send in two ordered lists of values, the populations from the wikipedia_data.html file and the win/loss ratio for a given sport in the same order. Average the win/loss ratios for those cities which have multiple teams of a single sport. Each sport is worth an equal amount in this assignment (20%\*4=80%) of the grade for this assignment. We will only use data **from year 2018** for our analysis.

### Part 1
For the first part, I calculate the win/loss ratio's correlation with the population of the city it is in for the **NHL** using **2018** data.

In [10]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re

In [51]:
def nhl_correlation():
    "Read in the data"
    nhl_df = pd.read_csv("assets/nhl.csv")
    cities = pd.read_html("assets/wikipedia_data.html")[1]
    cities = cities.iloc[:-1,[0,3,5,6,7,8]]
    
    "Clearing the data"
    nhl_df.team = nhl_df.team.map(lambda x: x.replace('*', ''))
    pattern_for_cities = r"(\[.*|\—)"
    cities.NHL = cities.NHL.map(lambda x: re.sub(pattern_for_cities, '', x))
    cities = cities.loc[(cities.NHL != '')]
    
    "Forming the columns that we will use"
    city_list = cities['Metropolitan area']
    pattern = '|'.join(city_list)
    nhl_df['Metropolitan area'] = nhl_df['team'].str.findall(pattern)
    nhl_df['Metropolitan area'] = [' '.join(map(str, x)) for x in nhl_df['Metropolitan area']]

    "Adding the missing values"
    nhl_df.loc[nhl_df['team'].str.contains('Tampa'), 'Metropolitan area'] = 'Tampa Bay Area'
    nhl_df.loc[nhl_df['team'].str.contains('Washington Capitals'), 'Metropolitan area'] = 'Washington, D.C.'
    nhl_df.loc[nhl_df['team'].str.contains('Carolina Hurricanes'), 'Metropolitan area'] = 'Raleigh'
    nhl_df.loc[nhl_df['team'].str.contains('New York'), 'Metropolitan area'] = 'New York City'
    nhl_df.loc[nhl_df['team'].str.contains('Minnesota Wild'), 'Metropolitan area'] = 'Minneapolis–Saint Paul'
    nhl_df.loc[nhl_df['team'].str.contains('Colorado Avalanche'), 'Metropolitan area'] = 'Denver'
    nhl_df.loc[nhl_df['team'].str.contains('New Jersey Devils'), 'Metropolitan area'] = 'New York City'
    nhl_df.loc[nhl_df['team'].str.contains('Dallas'), 'Metropolitan area'] = 'Dallas-Fort Worth'
    nhl_df.loc[nhl_df['team'].str.contains('Vegas'), 'Metropolitan area'] = 'Las Vegas'
    nhl_df.loc[nhl_df['team'].str.contains('Anaheim'), 'Metropolitan area'] = 'Los Angeles'
    nhl_df.loc[nhl_df['team'].str.contains('San Jose Sharks'), 'Metropolitan area'] = 'San Francisco Bay Area'
    nhl_df.loc[nhl_df['team'].str.contains('Arizona Coyotes'), 'Metropolitan area'] = 'Phoenix'
    nhl_df.loc[nhl_df['team'].str.contains('Florida Panthers'), 'Metropolitan area'] = 'Miami–Fort Lauderdale'
    
    "Removing the rows that have non-integer values and only the year 2018"
    nhl_df = nhl_df.loc[(nhl_df.team != 'Metropolitan Division') & (nhl_df.team != 'Atlantic Division') &\
                       (nhl_df.team != 'Central Division') & (nhl_df.team != 'Pacific Division') &\
                       (nhl_df.year == 2018)]
    
    "Changing to the int"
    nhl_df.W = nhl_df.W.astype(int)
    nhl_df.L = nhl_df.L.astype(int)

    "Actual analysis"
    group_nhl_df = nhl_df.groupby(by = ['Metropolitan area']).sum()
    group_nhl_df['win_loss_ratio'] = group_nhl_df['W'] / (group_nhl_df['W'] + group_nhl_df['L'])
    
    cities.sort_values(by = ['Metropolitan area'], inplace=True)
    
    win_loss_by_region = group_nhl_df['win_loss_ratio']
    population_by_region = cities['Population (2016 est.)[8]'].astype(int)
    
    corr, pval = stats.pearsonr(population_by_region, win_loss_by_region)
    
    assert len(population_by_region) == len(win_loss_by_region), "Lists must be the same length"
    assert len(population_by_region) == 28, "There should be 28 teams being analysed for NHL"
    return corr
    raise NotImplementedError()

nhl_correlation()

0.012308996455744264

### Part 2
For this part, I calculate the win/loss ratio's correlation with the population of the city it is in for the **NBA** using **2018** data.

In [56]:
def nba_correlation():
    "Read in the data"
    nba_df=pd.read_csv("assets/nba.csv")
    cities=pd.read_html("assets/wikipedia_data.html")[1]
    cities=cities.iloc[:-1,[0,3,5,6,7,8]]

    "Clearing the data"
    nba_df = nba_df.loc[(nba_df.year == 2018)]
    pattern_for_team = r"(\*)|\(.*\)"
    nba_df.team = nba_df.team.map(lambda x: re.sub(pattern_for_team, '', x))
    
    pattern_for_cities = r"(\[.*|\—)"
    cities.NBA = cities.NBA.map(lambda x: re.sub(pattern_for_cities, '', x))
    cities = cities.loc[(cities.NBA != '')]

    "Forming the columns that we will use"
    city_list = cities['Metropolitan area']
    pattern = '|'.join(city_list)
    nba_df['Metropolitan area'] = nba_df['team'].str.findall(pattern)
    nba_df['Metropolitan area'] = [' '.join(map(str, x)) for x in nba_df['Metropolitan area']]
    
    "Adding the missing values"
    nba_df.loc[nba_df['team'].str.contains('Indiana Pacers'), 'Metropolitan area'] = 'Indianapolis'
    nba_df.loc[nba_df['team'].str.contains('Miami Heat'), 'Metropolitan area'] = 'Miami–Fort Lauderdale'
    nba_df.loc[nba_df['team'].str.contains('Washington Wizards'), 'Metropolitan area'] = 'Washington, D.C.'
    nba_df.loc[nba_df['team'].str.contains('New York'), 'Metropolitan area'] = 'New York City'
    nba_df.loc[nba_df['team'].str.contains('Brooklyn Nets'), 'Metropolitan area'] = 'New York City'
    nba_df.loc[nba_df['team'].str.contains('Golden State Warriors'), 'Metropolitan area'] = 'San Francisco Bay Area'
    nba_df.loc[nba_df['team'].str.contains('Utah Jazz'), 'Metropolitan area'] = 'Salt Lake City'
    nba_df.loc[nba_df['team'].str.contains('Minnesota Timberwolves'), 'Metropolitan area'] = 'Minneapolis–Saint Paul'
    nba_df.loc[nba_df['team'].str.contains('Dallas Mavericks'), 'Metropolitan area'] = 'Dallas-Fort Worth'
    
    "Changing to the int"
    nba_df.W = nba_df.W.astype(int)
    nba_df.L = nba_df.L.astype(int)

    "Actual analysis"
    group_nba_df = nba_df.groupby(by = ['Metropolitan area']).sum()
    group_nba_df['win_loss_ratio'] = group_nba_df['W'] / (group_nba_df['W'] + group_nba_df['L'])
    
    cities.sort_values(by = ['Metropolitan area'], inplace=True)

    win_loss_by_region = group_nba_df['win_loss_ratio']
    population_by_region = cities['Population (2016 est.)[8]'].astype(int)

    corr, pval = stats.pearsonr(population_by_region, win_loss_by_region)
    
    assert len(population_by_region) == len(win_loss_by_region), "Lists must be the same length"
    assert len(population_by_region) == 28, "There should be 28 teams being analysed for NBA"

    return corr

nba_correlation()

-0.17657160252844611

### Part 3
For this part, I calculate the win/loss ratio's correlation with the population of the city it is in for the **MLB** using **2018** data.

In [57]:
def mlb_correlation(): 
    "Read in the data"
    mlb_df=pd.read_csv("assets/mlb.csv")
    cities=pd.read_html("assets/wikipedia_data.html")[1]
    cities=cities.iloc[:-1,[0,3,5,6,7,8]]
    
    "Clearing the data"
    mlb_df = mlb_df.loc[(mlb_df.year == 2018)]
    
    pattern_for_cities = r"(\[.*|\—)"
    cities.MLB = cities.MLB.map(lambda x: re.sub(pattern_for_cities, '', x))
    cities = cities.loc[(cities.MLB != '')]

    "Forming the columns that we will use"
    city_list = cities['Metropolitan area']
    pattern = '|'.join(city_list)
    mlb_df['Metropolitan area'] = mlb_df['team'].str.findall(pattern)
    mlb_df['Metropolitan area'] = [' '.join(map(str, x)) for x in mlb_df['Metropolitan area']]
    
    "Adding the missing values"
    mlb_df.loc[mlb_df['team'].str.contains('New York Yankees'), 'Metropolitan area'] = 'New York City'
    mlb_df.loc[mlb_df['team'].str.contains('Tampa Bay Rays'), 'Metropolitan area'] = 'Tampa Bay Area'
    mlb_df.loc[mlb_df['team'].str.contains('Minnesota Twins'), 'Metropolitan area'] = 'Minneapolis–Saint Paul'
    mlb_df.loc[mlb_df['team'].str.contains('Oakland Athletics'), 'Metropolitan area'] = 'San Francisco Bay Area'
    mlb_df.loc[mlb_df['team'].str.contains('Texas Rangers'), 'Metropolitan area'] = 'Dallas-Fort Worth'
    mlb_df.loc[mlb_df['team'].str.contains('Atlanta Braves'), 'Metropolitan area'] = 'Atlanta'
    mlb_df.loc[mlb_df['team'].str.contains('Washington Nationals'), 'Metropolitan area'] = 'Washington, D.C.'
    mlb_df.loc[mlb_df['team'].str.contains('New York Mets'), 'Metropolitan area'] = 'New York City'
    mlb_df.loc[mlb_df['team'].str.contains('Miami Marlins'), 'Metropolitan area'] = 'Miami–Fort Lauderdale'
    mlb_df.loc[mlb_df['team'].str.contains('Colorado Rockies'), 'Metropolitan area'] = 'Denver'
    mlb_df.loc[mlb_df['team'].str.contains('Arizona Diamondbacks'), 'Metropolitan area'] = 'Phoenix'
    mlb_df.loc[mlb_df['team'].str.contains('San Francisco Giants'), 'Metropolitan area'] = 'San Francisco Bay Area'
    
    "Changing to the int"
    mlb_df.W = mlb_df.W.astype(int)
    mlb_df.L = mlb_df.L.astype(int)

    "Actual analysis"
    group_mlb_df = mlb_df.groupby(by = ['Metropolitan area']).sum()
    group_mlb_df['win_loss_ratio'] = group_mlb_df['W'] / (group_mlb_df['W'] + group_mlb_df['L'])
    
    cities.sort_values(by = ['Metropolitan area'], inplace=True)

    win_loss_by_region = group_mlb_df['win_loss_ratio']
    population_by_region = cities['Population (2016 est.)[8]'].astype(int)

    corr, pval = stats.pearsonr(population_by_region, win_loss_by_region)

    assert len(population_by_region) == len(win_loss_by_region), "Lists must be the same length"
    assert len(population_by_region) == 26, "There should be 26 teams being analysed for MLB"

    return corr

mlb_correlation()

0.1505230448710485

### Part 4
For this part, I calculate the win/loss ratio's correlation with the population of the city it is in for the **NFL** using **2018** data.

In [75]:
def nfl_correlation():
    "Read in the data"
    nfl_df=pd.read_csv("assets/nfl.csv")
    cities=pd.read_html("assets/wikipedia_data.html")[1]
    cities=cities.iloc[:-1,[0,3,5,6,7,8]]
    
    "Clearing the data"
    pattern_for_team = r"(\+)|(\*)"
    nfl_df.team = nfl_df.team.map(lambda x: re.sub(pattern_for_team, '', x))
            
    pattern_for_cities = r"(\[.*|\—)"
    cities.NFL = cities.NFL.map(lambda x: re.sub(pattern_for_cities, '', x))
    cities = cities.loc[(cities.NFL != '') & (cities.NFL != ' ')]
    
    nfl_df = nfl_df.loc[(nfl_df.team != 'AFC East') & (nfl_df.team != 'AFC South') &\
                       (nfl_df.team != 'NFC East') & (nfl_df.team != 'AFC West') &\
                        (nfl_df.team != 'NFC North') & (nfl_df.team != 'NFC South') &\
                       (nfl_df.team != 'NFC West') & (nfl_df.team != 'AFC North') &\
                        (nfl_df.year == 2018)]
    


    "Forming the columns that we will use"
    city_list = cities['Metropolitan area']
    pattern = '|'.join(city_list)
    nfl_df['Metropolitan area'] = nfl_df['team'].str.findall(pattern)
    nfl_df['Metropolitan area'] = [' '.join(map(str, x)) for x in nfl_df['Metropolitan area']]
            
    "Adding the missing values"
    nfl_df.loc[nfl_df['team'].str.contains('New England Patriots'), 'Metropolitan area'] = 'Boston'
    nfl_df.loc[nfl_df['team'].str.contains('Miami Dolphins'), 'Metropolitan area'] = 'Miami–Fort Lauderdale'
    nfl_df.loc[nfl_df['team'].str.contains('New York Jets'), 'Metropolitan area'] = 'New York City'
    nfl_df.loc[nfl_df['team'].str.contains('Tennessee Titans'), 'Metropolitan area'] = 'Nashville'
    nfl_df.loc[nfl_df['team'].str.contains('Oakland Raiders'), 'Metropolitan area'] = 'San Francisco Bay Area'
    nfl_df.loc[nfl_df['team'].str.contains('Dallas Cowboys'), 'Metropolitan area'] = 'Dallas-Fort Worth'
    nfl_df.loc[nfl_df['team'].str.contains('Washington Redskins'), 'Metropolitan area'] = 'Washington, D.C.'
    nfl_df.loc[nfl_df['team'].str.contains('New York Giants'), 'Metropolitan area'] = 'New York City'
    nfl_df.loc[nfl_df['team'].str.contains('Minnesota Vikings'), 'Metropolitan area'] = 'Minneapolis–Saint Paul'
    nfl_df.loc[nfl_df['team'].str.contains('Carolina Panthers'), 'Metropolitan area'] = 'Charlotte'
    nfl_df.loc[nfl_df['team'].str.contains('Tampa Bay Buccaneers'), 'Metropolitan area'] = 'Tampa Bay Area'
    nfl_df.loc[nfl_df['team'].str.contains('San Francisco 49ers'), 'Metropolitan area'] = 'San Francisco Bay Area'
    nfl_df.loc[nfl_df['team'].str.contains('Arizona Cardinals'), 'Metropolitan area'] = 'Phoenix'
    
    "Changing to the int"
    nfl_df.W = nfl_df.W.astype(int)
    nfl_df.L = nfl_df.L.astype(int)

    "Actual analysis"
    group_nfl_df = nfl_df.groupby(by = ['Metropolitan area']).sum()
    group_nfl_df['win_loss_ratio'] = group_nfl_df['W'] / (group_nfl_df['W'] + group_nfl_df['L'])
    
    cities.sort_values(by = ['Metropolitan area'], inplace=True)

    win_loss_by_region = group_nfl_df['win_loss_ratio']
    population_by_region = cities['Population (2016 est.)[8]'].astype(int)

    corr, pval = stats.pearsonr(population_by_region, win_loss_by_region)

    assert len(population_by_region) == len(win_loss_by_region), "Lists must be the same length"
    assert len(population_by_region) == 29, "There should be 29 teams being analysed for NFL"    
    return corr

nfl_correlation()

0.004922112149349456